In [1]:
from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [8]:
from langchain.schema import SystemMessage, HumanMessage
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature = 0, model="gpt-3.5-turbo")
message = """
In ChatOpenAI, there are three types of messages: HumanMessage, SystemMessage, and AIMessage.

HumanMessage is a message sent from the perspective of the human. It can be anything that the human wants to say to the AI, such as a question, a request, or a statement.
SystemMessage is a message setting the objectives the AI should follow. It is sent from the system to the AI, and it can be used to give the AI instructions or to provide feedback.
AIMessage is a message sent from the perspective of the AI. It can be anything that the AI wants to say to the human, such as an answer, a suggestion, or a clarification.
The other one you mentioned is ChatMessage, which is a message allowing for arbitrary setting of role. It is a subclass of HumanMessage and AIMessage."""
text = [
    SystemMessage(content="You are an expert copy writer with expertise in summarizing content"), 
    HumanMessage(content=f"Please provide a short and concise summary of the following,  Text:{message}")
]

In [4]:
llm.get_num_tokens(message)

364

In [5]:
summary = llm(text)

In [6]:
summary.content

'The temperature in ChatOpenAI controls the randomness of generated text. A higher temperature leads to more creative and diverse text, while a lower temperature results in more conservative and predictable text. The temperature value ranges from 0 to 1, with 0 always choosing the most likely word and 1 choosing any word with equal probability. For creative tasks, a temperature of 0.7 or higher is recommended, while for factual tasks, a temperature of 0.4 or lower is preferred. For a mix of creativity and factuality, a temperature of 0.5 or 0.6 can be used. Experimentation with different temperature settings is encouraged to find the best fit.'

In [7]:
from langchain import PromptTemplate
from langchain.chains import LLMChain

In [13]:
template = ("Write a short and concise summary of the following text"
            "Text: {text}. Translate the summary to {language}")
prompt = PromptTemplate(
    input_variables = ["text", "language"],
    template=template
)
chain = LLMChain(llm= llm, prompt=prompt)
summary = chain.run({"text": message, 'language': "hindi"})
summary

'ChatOpenAI में तीन प्रकार के संदेश होते हैं: HumanMessage, SystemMessage और AIMessage. HumanMessage मानव के दृष्टिकोण से भेजा गया संदेश होता है। यह कुछ भी हो सकता है जो मानव एआई को कहना चाहता है, जैसे सवाल, अनुरोध या कथन। SystemMessage एआई को अनुसरण करने के लिए उद्देश्य सेट करने वाला संदेश होता है। यह सिस्टम से एआई को निर्देश देने या प्रतिक्रिया प्रदान करने के लिए उपयोग किया जा सकता है। AIMessage एआई के दृष्टिकोण से भेजा गया संदेश होता है। यह कुछ भी हो सकता है जो एआई मानव को कहना चाहता है, जैसे जवाब, सुझाव या स्पष्टीकरण। आपने उल्लेख किया है वह एक और है ChatMessage, जो भूमिका की विचारशील सेटिंग की अनुमति देता है। यह HumanMessage और AIMessage का एक उपप्रकार है।'

In [21]:
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document

with open('../files/sj.txt', encoding='utf-8') as f:
    sj_message = f.read()
docs = [Document(page_content= sj_message)]
template = ("Write a short and concise summary of the following text"
            "Text: {text}. ")
prompt = PromptTemplate(
    input_variables = ["text"],
    template=template
)
chain = load_summarize_chain(
    llm,
    chain_type = 'stuff',
    prompt = prompt,
    verbose=False
)
chain.run(docs)

'The speaker, who never graduated from college, shares three stories from his life during a commencement speech. The first story is about the importance of connecting the dots and how dropping out of college led him to take a calligraphy class that later influenced the design of the Macintosh computer. The second story is about love and loss, as he reflects on being fired from the company he co-founded and how it ultimately led to new opportunities and personal growth. The third story is about death, as he recounts his experience with a cancer diagnosis and emphasizes the importance of living life to the fullest. The speaker encourages the graduates to follow their passions, trust their intuition, and stay hungry and foolish in their pursuit of success.'

In [24]:
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

with open('../files/sj.txt', encoding='utf-8') as f:
    sj_message = f.read()

splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=50)
# docs = [Document(page_content= sj_message)]
chunks = splitter.create_documents([sj_message])
template = ("Write a short and concise summary of the following text"
            "Text: {text}. ")
# prompt = PromptTemplate(
#     input_variables = ["text"],
#     template=template
# )
chain = load_summarize_chain(
    llm,
    chain_type = 'map_reduce',
    verbose=False
)
chain.run(chunks)

"Steve Jobs shares three stories from his life, including dropping out of college and how it influenced the design of the Macintosh computer, getting fired from Apple and finding success in new ventures, and his experience with cancer and the importance of living each day fully. He emphasizes the importance of following one's passion, not settling, and embracing the inevitability of death. Jobs encourages the audience to live their own lives, follow their hearts, and mentions The Whole Earth Catalog as an inspiration. He concludes by wishing the audience well on their new beginnings."

In [26]:
 chain.llm_chain.prompt.template


'Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:'

In [27]:
chain.combine_document_chain.llm_chain.prompt.template

'Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:'

In [74]:

from langchain.document_loaders import UnstructuredPDFLoader
loader = UnstructuredPDFLoader("../files/attention_is_all_you_need.pdf")
kenya_cons = loader.load()

In [80]:
splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=5)
chunks = splitter.split_documents(kenya_cons)
def print_embedding_cost(texts):
    import tiktoken
    enc = tiktoken.encoding_for_model('gpt-3.5-turbo')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    print(f'Total tokens: {total_tokens}')
    print(f'Embedding cost in USD: {total_tokens / 1000 * 0.0004:.6f}')

for chunk in chunks:
    print(len(chunk.page_content))
print_embedding_cost(chunks)


9762
9993
9801
9994
466
Total tokens: 9995
Embedding cost in USD: 0.003998


In [81]:
chain = load_summarize_chain(
    llm,
    chain_type='refine',
    verbose=False
)
summary = chain.run(chunks)

In [82]:
summary

'The paper introduces the Transformer, a network architecture based solely on attention mechanisms, which achieves superior results in machine translation tasks compared to existing models. The Transformer is more parallelizable and requires less training time. The paper discusses the advantages of self-attention and describes the architecture of the Transformer, including the encoder and decoder stacks and the attention mechanism used. The paper also compares self-attention to recurrent and convolutional layers in terms of computational complexity, parallelizability, and the ability to learn long-range dependencies. The authors propose using positional encodings to inject information about the relative or absolute position of tokens in the sequence. The paper concludes by discussing the training process and the potential interpretability of self-attention models. Additionally, the paper provides details on the training data, hardware, schedule, optimizer, and regularization techniques

In [83]:
map_prompt = '''
Write a short and concise summary of the following:
Text: `{text}`
CONCISE SUMMARY:
'''
map_prompt_template = PromptTemplate(
    input_variables=['text'],
    template=map_prompt
)

In [84]:
combine_prompt = '''
Write a concise summary of the following text that covers the key points.
Add a title to the summary.
Start your summary with an INTRODUCTION PARAGRAPH that gives an overview of the topic FOLLOWED
by BULLET POINTS if possible AND end the summary with a CONCLUSION PHRASE.
Text: `{text}`
'''
combine_prompt_template = PromptTemplate(template=combine_prompt, input_variables=['text'])

In [85]:
summary_chain = load_summarize_chain(
    llm=llm,
    chain_type='map_reduce',
    map_prompt=map_prompt_template,
    combine_prompt=combine_prompt_template,
    verbose=False
)
summary_chain.run(chunks)

"Title: The Transformer: A New Network Architecture Based on Attention Mechanisms\n\nIntroduction:\nThe paper introduces the Transformer, a network architecture that relies solely on attention mechanisms and eliminates the need for recurrent or convolutional neural networks. The Transformer achieves superior results in machine translation tasks, offers parallelizability, and requires less training time compared to existing models.\n\nKey Points:\n- The Transformer model utilizes self-attention and fully connected layers in both the encoder and decoder.\n- The attention mechanism used in the Transformer is called Scaled Dot-Product Attention.\n- The text compares additive attention and dot-product attention, with the latter being faster and more space-efficient.\n- Multi-head attention allows the model to attend to different information simultaneously.\n- Self-attention layers and positional encodings capture sequence order and outperform recurrent and convolutional layers in terms of c

In [86]:
prompt_template = """Write a concise summary of the following extracting the key information:
Text: `{text}`
CONCISE SUMMARY:"""
initial_prompt = PromptTemplate(template=prompt_template, input_variables=['text'])

refine_template = '''
    Your job is to produce a final summary.
    I have provided an existing summary up to a certain point: {existing_answer}.
    Please refine the existing summary with some more context below.
    ------------
    {text}
    ------------
    Start the final summary with an INTRODUCTION PARAGRAPH that gives an overview of the topic FOLLOWED
    by BULLET POINTS if possible AND end the summary with a CONCLUSION PHRASE.
    
'''
refine_prompt = PromptTemplate(
    template=refine_template,
    input_variables=['existing_answer', 'text']
)


In [87]:
chain = load_summarize_chain(
    llm=llm,
    chain_type='refine',
    question_prompt=initial_prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=False
    
)
output_summary = chain.run(chunks)

In [88]:
output_summary

"The paper introduces the Transformer, a network architecture that relies solely on attention mechanisms and eliminates the need for recurrent or convolutional neural networks. The Transformer achieves superior results in machine translation tasks, offers better parallelization capabilities, and requires less training time compared to existing models. It also demonstrates state-of-the-art performance in translation quality and generalizes well to other tasks. The paper provides a detailed description of the Transformer's architecture, including the encoder and decoder stacks, as well as the attention mechanism used. The training regime for the models is also discussed, including the training data, hardware and schedule, optimizer, and regularization techniques. The Transformer architecture, based solely on attention mechanisms, offers significant advantages in terms of performance, parallelization, and training time compared to existing models. By employing multi-head attention and pos

In [89]:
from langchain.utilities import WikipediaAPIWrapper
from langchain.agents import initialize_agent, Tool
wikipedia = WikipediaAPIWrapper()
tools = [
    Tool(
        name="Wikipedia", 
        func=wikipedia.run,
        description="Useful for when you need to get information from wikipedia about a single topic"
    )
]

In [90]:
agent_executor = initialize_agent(tools, llm, agent='zero-shot-react-description', verbose=True)

In [92]:
agent_executor.run('Can you please provide a short summary of the state of taxes currently in kenya')



> Entering new AgentExecutor chain...
I should use Wikipedia to find information about the state of taxes in Kenya.
Action: Wikipedia
Action Input: "Taxes in Kenya"
Observation: Page: Kenya Revenue Authority
Summary: Kenya Revenue Authority (KRA), is an agency of the Government of Kenya that is responsible for the assessment, collection and accounting of all revenues that are due to the government in accordance with the laws of Kenya.

Page: Kenya
Summary: Kenya, officially the Republic of Kenya (Swahili: Jamhuri ya Kenya), is a country in East Africa. A member of the Commonwealth with a population of more than 47.6 million in the 2019 census, Kenya is the 28th most populous country in the world and 7th most populous in Africa. Kenya's capital and largest city is Nairobi, while its oldest and second largest city, which until 1907 was also Kenya's first capital city, is the coastal city of Mombasa which includes Mombasa Island in the Indian Ocean and the surrounding mainland. Kisumu i

'The state of taxes in Kenya is governed by the Kenya Revenue Authority (KRA), which is responsible for the assessment, collection, and accounting of all revenues due to the government. The taxation system in Kenya covers income tax, value-added tax, customs, and excise duty.'